In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Importing Libraries
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

2023-02-14 09:28:53.708874: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 09:28:54.084756: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-14 09:28:54.084829: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-14 09:28:56.354300: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
# Data generation for training data
train_data = ImageDataGenerator(rescale = 1./255)

In [4]:
validation_data = ImageDataGenerator(rescale=1./255)

In [5]:
# Training Dataset
train = train_data.flow_from_directory(directory ='/home/ubuntu/Desktop/sep_data_final/sep_data/train/',
                                       target_size=(224, 224),
                                       batch_size = 16,
                                       class_mode ="categorical")

Found 10880 images belonging to 36 classes.


In [6]:
#validation dataset
validation = validation_data.flow_from_directory(directory ='/home/ubuntu/Desktop/sep_data_final/sep_data/val/',
                                                 target_size=(224, 224),
                                                 batch_size = 16,
                                                 class_mode ="categorical")
                                                 

Found 2050 images belonging to 36 classes.


In [7]:
base_model = keras.applications.ResNet50V2(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

2023-02-14 09:28:59.950844: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-14 09:28:59.950920: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-14 09:28:59.950958: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-Zako): /proc/driver/nvidia/version does not exist
2023-02-14 09:28:59.951314: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
base_model.trainable = False

In [9]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

#pooling layer 
x = keras.layers.GlobalAveragePooling2D()(x)

#final dense layer
outputs = keras.layers.Dense(36, activation = 'softmax')(x)

model = keras.Model(inputs,outputs)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 36)                73764     
                                                                 
Total params: 23,638,564
Trainable params: 73,764
Non-trainable params: 23,564,800
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',loss=['categorical_crossentropy'],metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
# Early stopping to avoid overfitting
es = EarlyStopping(monitor = "accuracy",patience = 3)

In [13]:
model.fit(train, validation_data = validation, epochs = 10, callbacks = [es])

Epoch 1/10
680/680 [==============================] - 2824s 4s/step - loss: 0.6431 - accuracy: 0.8459 - val_loss: 0.3838 - val_accuracy: 0.9200
Epoch 2/10
680/680 [==============================] - 2641s 4s/step - loss: 0.1812 - accuracy: 0.9577 - val_loss: 0.3242 - val_accuracy: 0.9317
Epoch 3/10
680/680 [==============================] - 2378s 3s/step - loss: 0.1035 - accuracy: 0.9778 - val_loss: 0.3053 - val_accuracy: 0.9380
Epoch 4/10
680/680 [==============================] - 2751s 4s/step - loss: 0.0671 - accuracy: 0.9881 - val_loss: 0.2953 - val_accuracy: 0.9434
Epoch 5/10
680/680 [==============================] - 2573s 4s/step - loss: 0.0456 - accuracy: 0.9937 - val_loss: 0.2800 - val_accuracy: 0.9429
Epoch 6/10
680/680 [==============================] - 2331s 3s/step - loss: 0.0335 - accuracy: 0.9956 - val_loss: 0.2953 - val_accuracy: 0.9439
Epoch 7/10
680/680 [==============================] - 2362s 3s/step - loss: 0.0257 - accuracy: 0.9970 - val_loss: 0.2954 - val_accuracy:

In [14]:
model.save("Model.h5")